# Calculate Expected Annual Damage for all road segments in a NUTS-3 region.
This script calls the EAD_multi script, which carries out the computation of the Expected Annual Damage using (1) the output of the main model and (2) the flood protection data. Before running this notebook, make sure to have run the main model as well as the postprocessing AoI_FP step.

In [1]:
import sys
sys.path.append("..") #import folder which is one level higher

import os as os
import pandas as pd
from pathos.multiprocessing import Pool
import time as time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from postproc_functions import EAD_multi, NUTS_up, NUTS_0_list
from utils_functions import load_config

### Run for one NUTS-3 region

In [2]:
EAD_multi('DE224')
#output = EAD_multi('AL013')

EAD_DE224: 100%|███████████████████████████████████████████████████████████████████| 4969/4969 [02:16<00:00, 36.49it/s]



 DE224: calculation of EAD successfull


,infra_type,geometry,lanes,bridge,lit,length,road_type,length_rp10,val_rp10,length_rp20,...,Jongman_FP,EAD_C1,EAD_C2,EAD_C3,EAD_C4,EAD_C5,EAD_C6,EAD_HZ,EAD_lowflow,EAD_highflow
osm_id,,,,,,,,,,,,,,,,,,,,,
4019778,motorway,"LINESTRING (13.04178 48.77036, 13.04256 48.77006)",3.0,NaN,NaN,0.066293,motorway,0.004590,78,0.004590,...,150.0,107.722247,215.444494,14.816479,144.77633,0.000000,0.000000,100.236704,14.816479,144.776330
4051029,primary,"LINESTRING (12.92883 48.74648, 12.92995 48.74611)",2.0,NaN,no,0.091771,primary,0.020482,35,0.020482,...,150.0,0.000000,0.000000,0.000000,0.00000,10.100974,80.782022,22.701423,10.100974,80.782022
4051097,track,"LINESTRING (12.96312 48.81146, 12.96324 48.81119)",1.0,NaN,NaN,0.031360,track,0.001851,234,0.001851,...,150.0,0.000000,0.000000,0.000000,0.00000,0.025768,0.180375,1.082247,0.025768,0.180375
4051212,service,"LINESTRING (12.96016 48.82960, 12.95944 48.82891)",1.0,NaN,NaN,0.092915,other,0.092893,334,0.092893,...,150.0,0.000000,0.000000,0.000000,0.00000,7.459775,53.416629,59.678202,7.459775,53.416629
4051280,primary,"LINESTRING (12.87130 48.77968, 12.87128 48.779...",2.0,NaN,yes,0.088091,primary,0.088002,31,0.088002,...,150.0,0.000000,0.000000,0.000000,0.00000,36.319700,290.583371,76.504569,36.319700,290.583371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639916200,service,"LINESTRING (12.89787 48.78959, 12.89808 48.789...",1.0,NaN,NaN,0.116553,other,0.116553,149,0.116553,...,150.0,0.000000,0.000000,0.000000,0.00000,7.047491,56.148015,49.499925,7.047491,56.148015
639916202,service,"LINESTRING (12.89281 48.78799, 12.89298 48.787...",1.0,NaN,NaN,0.046346,other,0.045928,66,0.045927,...,150.0,0.000000,0.000000,0.000000,0.00000,1.919700,15.318951,12.059326,1.919700,15.318951
639916204,service,"LINESTRING (12.89259 48.78864, 12.89256 48.788...",1.0,NaN,NaN,0.068945,other,0.067674,166,0.067674,...,150.0,0.000000,0.000000,0.000000,0.00000,3.994007,31.952060,27.777678,3.994007,31.952060


### Select regions to run the model for

In [3]:
#RUN FOR ENTIRE EUROPE
N0_domain = NUTS_0_list(EU28 = True, EFTA=True, CAND=True)
N0_exclude = ['MT','CY','TR','IS']
N0_domain = [elem for elem in N0_domain if elem not in N0_exclude] #35 countries remaining
regions = []
for N0 in N0_domain:
    regions.extend(NUTS_up(N0,True)) #1519 NUTS-3 regions

#RUN FOR ONE COUNTRY
#regions = NUTS_up('CH',True)

#RUN FOR MULTIPLE COUNTRIES
#super_regions = ['LI','NO']
#regions = []
#for super_region in super_regions:
#    regions.extend(NUTS_up(super_region,True))

### Run for multiple regions using parallel processing

In [4]:
start = time.time()

if __name__ == '__main__': 
    with Pool(4) as pool: 
        pool.map(EAD_multi,regions,chunksize=1)
    
print("The script runned for {} seconds.".format(time.time()-start))

UnboundLocalError: local variable 'df2' referenced before assignment